In [1]:
import os
import torch
from ultralytics import YOLO
import numpy as np
from torch.nn import functional as F
import yaml
from tqdm import tqdm

# Configuración
DATASET_PATH = "/home/mosquitos/Documentos/TesisDiegoMathi/Mosquitos-1/"
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def setup_dataset():
    # Configurar el dataset
    data_yaml = os.path.join(DATASET_PATH, 'data.yaml')
    
    # Modificar el archivo YAML para establecer img_size
    with open(data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)
    
    data_config['imgsz'] = IMG_SIZE
    
    # Guardar configuración actualizada
    with open(data_yaml, 'w') as f:
        yaml.dump(data_config, f)
    
    return data_yaml

def two_stage_distillation():
    """
    Implementación de destilación en dos etapas:
    1. Entrenar el modelo student (nano) de forma normal
    2. Ajustar el student usando la destilación del teacher (small)
    """
    print("Cargando modelos teacher y student...")
    
    # Cargar modelos
    teacher = YOLO('best.pt')  # Modelo teacher (small)
    student = YOLO('yolov8n.pt')  # Modelo student (nano)
    
    # Configurar dataset
    data_yaml = setup_dataset()
    
    # ETAPA 1: Entrenamiento estándar del student
    print("\n==== ETAPA 1: Entrenamiento estándar del modelo student ====")
    student.train(
        data=data_yaml,
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        device=DEVICE,
        name='student_base'  # Nombre para los logs
    )
    
    # Guardar el modelo entrenado - corregido para usar los argumentos válidos
    student.export(format='torchscript', name='student_base')
    print("Modelo student base guardado como 'student_base.pt'")
    
    # Determinar la ruta donde se guardó el modelo student base
    run_dir = os.path.join('runs', 'detect', 'student_base')
    student_base_path = os.path.join(run_dir, 'weights', 'best.pt')
    if not os.path.exists(student_base_path):
        student_base_path = os.path.join(run_dir, 'weights', 'last.pt')
    
    # ETAPA 2: Ajuste fino con destilación
    print("\n==== ETAPA 2: Ajuste fino con destilación de conocimiento ====")
    
    # Cargar el modelo student entrenado
    student_trained = YOLO(student_base_path)
    
    # Crear un clon del dataset, pero con anotaciones generadas por el teacher
    create_teacher_dataset(teacher, data_yaml)
    
    # Entrenar nuevamente el student, pero usando las etiquetas del teacher
    student_trained.train(
        data=os.path.join(DATASET_PATH, 'teacher_data.yaml'),  # Dataset con anotaciones del teacher
        epochs=100,  # Menos épocas para el ajuste fino
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        device=DEVICE,
        name='student_distilled'  # Nombre para los logs
    )
    
    # Guardar el modelo destilado final - corregido
    student_trained.export(format='torchscript', name='yolov8n_distilled')
    print("Modelo student destilado guardado en la carpeta 'runs/detect/student_distilled/weights/'")
    
    return student_trained

def create_teacher_dataset(teacher, original_data_yaml):
    """
    Crea un nuevo dataset con anotaciones generadas por el modelo teacher
    """
    print("Creando dataset con anotaciones del teacher...")
    
    # Cargar la configuración original del dataset
    with open(original_data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)
    
    # Crear directorios para el nuevo dataset
    teacher_dataset_path = os.path.join(DATASET_PATH, 'teacher_dataset')
    os.makedirs(teacher_dataset_path, exist_ok=True)
    os.makedirs(os.path.join(teacher_dataset_path, 'images'), exist_ok=True)
    os.makedirs(os.path.join(teacher_dataset_path, 'labels'), exist_ok=True)
    
    # Copiar imágenes y generar nuevas etiquetas con el teacher
    from glob import glob
    import shutil
    from PIL import Image
    
    # Obtener las rutas de las imágenes de entrenamiento
    train_dir = data_config.get('train', '')
    
    # Manejar diferentes formatos de configuración YAML
    if isinstance(train_dir, list):
        train_dir = train_dir[0]
    
    # Ver si train_dir es una ruta directa a las imágenes o a un directorio que contiene un subdirectorio 'images'
    if os.path.exists(os.path.join(train_dir, 'images')):
        train_dir = os.path.join(train_dir, 'images')
    
    train_images = glob(os.path.join(train_dir, '*.jpg')) + \
                  glob(os.path.join(train_dir, '*.png')) + \
                  glob(os.path.join(train_dir, '*.jpeg'))
    
    if not train_images:
        # Intenta con subdirectorios
        train_images = glob(os.path.join(train_dir, '**', '*.jpg'), recursive=True) + \
                      glob(os.path.join(train_dir, '**', '*.png'), recursive=True) + \
                      glob(os.path.join(train_dir, '**', '*.jpeg'), recursive=True)
    
    print(f"Procesando {len(train_images)} imágenes con el teacher...")
    
    # Poner el teacher en modo evaluación
    teacher.model.eval()
    
    # Procesar cada imagen con el teacher
    for img_path in tqdm(train_images):
        # Nombre del archivo
        img_name = os.path.basename(img_path)
        base_name = os.path.splitext(img_name)[0]
        
        # Copiar la imagen al nuevo dataset
        new_img_path = os.path.join(teacher_dataset_path, 'images', img_name)
        shutil.copy(img_path, new_img_path)
        
        # Generar etiquetas con el teacher
        results = teacher(img_path, verbose=False)
        
        # Crear el archivo de etiquetas en formato YOLO
        label_path = os.path.join(teacher_dataset_path, 'labels', f"{base_name}.txt")
        
        # Obtener las detecciones y convertirlas a formato YOLO
        with open(label_path, 'w') as f:
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    # Clase, x_centro, y_centro, ancho, alto (normalizado)
                    cls = int(box.cls.item())
                    x, y, w, h = box.xywhn[0].tolist()  # Coordenadas normalizadas
                    conf = box.conf.item()  # Confianza
                    
                    # Solo incluir detecciones con alta confianza
                    if conf > 0.40: #0.25
                        f.write(f"{cls} {x} {y} {w} {h}\n")

    #TABLA CONFIANZA
    # Crear un nuevo archivo data.yaml para el dataset del teacher
    teacher_data_yaml = os.path.join(DATASET_PATH, 'teacher_data.yaml')
    
    # Actualizar rutas en la configuración
    data_config['train'] = os.path.join(teacher_dataset_path, 'images')
    
    # Asegurarse de que validación y test estén configurados correctamente
    if 'val' in data_config:
        val_dir = data_config['val']
        if isinstance(val_dir, list):
            val_dir = val_dir[0]
        if os.path.exists(os.path.join(val_dir, 'images')):
            data_config['val'] = os.path.join(val_dir, 'images')
    elif 'test' in data_config:
        data_config['val'] = data_config['test']
    
    # Guardar la nueva configuración
    with open(teacher_data_yaml, 'w') as f:
        yaml.dump(data_config, f)
    
    print(f"Dataset del teacher creado en {teacher_dataset_path}")
    print(f"Archivo de configuración guardado como {teacher_data_yaml}")
    
    return teacher_data_yaml

if __name__ == "__main__":
    student = two_stage_distillation()
    
    # Evaluar el modelo destilado
    # Determinar la ruta donde se guardó el modelo destilado
    distilled_path = os.path.join('runs', 'detect', 'student_distilled', 'weights', 'best.pt')
    if not os.path.exists(distilled_path):
        distilled_path = os.path.join('runs', 'detect', 'student_distilled', 'weights', 'last.pt')
    
    # Cargar y evaluar el modelo destilado
    student_eval = YOLO(distilled_path)
    metrics = student_eval.val()
    
    print(f"\nRendimiento del modelo student después de la destilación:")
    if hasattr(metrics, 'box'):
        print(f"mAP50: {metrics.box.map50:.4f}")
        print(f"mAP50-95: {metrics.box.map:.4f}")
    else:
        print(metrics)

Cargando modelos teacher y student...

==== ETAPA 1: Entrenamiento estándar del modelo student ====
New https://pypi.org/project/ultralytics/8.3.130 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/mosquitos/Documentos/TesisDiegoMathi/Mosquitos-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=student_base, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

E0000 00:00:1746808850.443881 2481442 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746808850.458169 2481442 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /home/mosquitos/Documentos/TesisDiegoMathi/Mosquitos-1/train/labels.cache... 2157 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2157/2157 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 519, len(boxes) = 6008. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

                   all        240        638      0.741      0.532      0.556      0.288
                Female        129        322      0.808      0.898      0.887      0.476
            Incomplete         14         15          1      0.119      0.278      0.126
                  Male        102        187      0.844      0.947      0.941      0.544
          Out of Focus         63         94      0.568      0.596      0.565      0.248
             Undefined         18         20      0.487        0.1      0.109     0.0467
Speed: 0.0ms preprocess, 0.4ms inference, 0.0ms loss, 5.0ms postprocess per image
Results saved to runs/detect/student_distilled
Ultralytics 8.3.116 🚀 Python-3.10.12 torch-2.6.0+cu124 CPU (Intel Xeon E5-2666 v3 2.90GHz)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/student_distilled/weights/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 9, 1029) (5.9 MB)

TorchScript:

val: Scanning /home/mosquitos/Documentos/TesisDiegoMathi/Mosquitos-1/valid/labels.cache... 240 images, 2 backgrounds, 0 corrupt: 100%|██████████| 240/240 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 30, len(boxes) = 638. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:02<00:00,  6.47it/s]


                   all        240        638      0.741      0.532      0.556      0.289
                Female        129        322      0.808      0.898      0.887      0.476
            Incomplete         14         15          1      0.119      0.278      0.126
                  Male        102        187      0.845      0.947      0.941      0.546
          Out of Focus         63         94      0.568      0.596      0.565      0.248
             Undefined         18         20      0.486        0.1      0.109     0.0467
Speed: 0.1ms preprocess, 5.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val

Rendimiento del modelo student después de la destilación:
mAP50: 0.5560
mAP50-95: 0.2886


In [3]:
import os
import torch
from ultralytics import YOLO
import numpy as np
from torch.nn import functional as F
import yaml
from tqdm import tqdm

In [14]:
base_path = os.path.join('runs', 'detect', 'student_base', 'weights', 'best.pt')
student_eval0 = YOLO(base_path)
metrics = student_eval0.val()

Ultralytics 8.3.116 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24250MiB)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1015.2±229.1 MB/s, size: 25.2 KB)


val: Scanning /home/mosquitos/Documentos/TesisDiegoMathi/Mosquitos-1/valid/labels.cache... 240 images, 2 backgrounds, 0 corrupt: 100%|██████████| 240/240 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 30, len(boxes) = 638. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:02<00:00,  6.11it/s]


                   all        240        638       0.59      0.547      0.589      0.285
                Female        129        322      0.802      0.848      0.871      0.449
            Incomplete         14         15      0.621        0.4      0.425      0.147
                  Male        102        187      0.862      0.893      0.939      0.524
          Out of Focus         63         94      0.665      0.596      0.657      0.281
             Undefined         18         20          0          0     0.0509      0.025
Speed: 0.1ms preprocess, 4.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val35


In [23]:
base_path = os.path.join('runs', 'detect', 'student_distilled', 'weights', 'best.pt')
student_eval0 = YOLO(base_path)
metrics = student_eval0.val()

Ultralytics 8.3.116 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24250MiB)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 961.4±259.4 MB/s, size: 26.6 KB)


val: Scanning /home/mosquitos/Documentos/TesisDiegoMathi/Mosquitos-1/valid/labels.cache... 240 images, 2 backgrounds, 0 corrupt: 100%|██████████| 240/240 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 30, len(boxes) = 638. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:02<00:00,  6.11it/s]


                   all        240        638      0.741      0.532      0.556      0.289
                Female        129        322      0.808      0.898      0.887      0.476
            Incomplete         14         15          1      0.119      0.278      0.126
                  Male        102        187      0.845      0.947      0.941      0.546
          Out of Focus         63         94      0.568      0.596      0.565      0.248
             Undefined         18         20      0.486        0.1      0.109     0.0467
Speed: 0.1ms preprocess, 4.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val44
